<a href="https://colab.research.google.com/github/AshvinVignesh/Final_year/blob/main/final_year_LR_and_sampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install deap

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 3.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np
from sklearn.model_selection import cross_val_score, KFold ,train_test_split ,GridSearchCV,cross_validate
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report , f1_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder , FunctionTransformer
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
# from deap import base, creator, tools, algorithms
import random
from sklearn.pipeline import Pipeline

In [ ]:
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
# from imblearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
#from deap import base, creator, tools, algorithms
import random

In [ ]:
df = pd.read_csv("Full_data_aspect_removed_words_Final.csv")
df.head()

,original_text,sentences_with_pos_tags,lang,sentiment,clean_text,Acting,direction,Music,VFX,Genre,excitement,ovr_sent,stop_words
0,இரைச்சல் இல்லாத தெளிவான இசை,இரைச்சல்/NOUN இல்லாத/ADJ தெளிவான/ADJ இசை/NOUN,Tamil,POSITIVE,இரைச்சல் இல்லாத தெளிவான இசை,0,0,1,0,0,0,1,"இல்லாத, தெளிவான"
1,harris romba naal kalichu super song,harris/PROPN romba/ADV naal/NOUN kalichu/VERB ...,Code-Mixed,POSITIVE,harris romba naal kalichu super song,0,0,1,0,0,0,1,"romba, naal, kalichu"
2,hukum pondra iraichal padalai ketu kaathu vali...,hukum/PROPN pondra/ADP iraichal/NOUN padalai/N...,Code-Mixed,NEUTRAL,hukum pondra iraichal padalai ketu kaathu vali...,0,0,1,0,0,0,1,"pondr, iraichal, ketu, kaathu, valichuruchu, a..."
3,youtube la kuda editing nalla pannuvanga,youtube/PROPN la/ADP kuda/ADV editing/NOUN nal...,Code-Mixed,NEUTRAL,youtube kuda editing nalla pannuvanga,0,0,0,1,0,0,1,"la, kuda"
4,amaran is also a diwali release,amaran/PROPN is/AUX also/ADV a/DET diwali/PROP...,Code-Mixed,NEUTRAL,amaran diwali release,0,0,0,0,0,0,0,"is, also, a"


In [ ]:
X = df["clean_text"]
y = df[['Acting', 'direction', 'Music',  'Genre', 'excitement','ovr_sent']] # Use a list of column names to select multiple columns

In [ ]:
def prepare_data(X, y_df):
    # Convert text to TF-IDF features
    # Handle missing values by replacing them with an empty string
    X = X.fillna('')
    vectorizer = TfidfVectorizer(max_features=1000)
    X_tfidf = vectorizer.fit_transform(X)

    # Prepare label encoders for each target
    label_encoders = {}
    y_encoded = pd.DataFrame()

    # Encode each target column
    for column in y_df.columns:
        le = LabelEncoder()
        y_encoded[column] = le.fit_transform(y_df[column].astype(str))
        label_encoders[column] = le

    return X_tfidf, y_encoded, label_encoders

def custom_f1_score(y_true, y_pred):
    return f1_score(y_true, y_pred, average='micro')

In [ ]:
C_RANGE = [0.01, 0.1, 1.0, 10.0, 100.0]
PENALTY_RANGE = ['l1', 'l2']  # Use solvers that support l1 and l2 (e.g., 'liblinear', 'saga')
SOLVER_RANGE = ['liblinear', 'saga']
MAX_ITER_RANGE = [100, 200, 300, 500]

In [ ]:
def custom_mutation(individual, indpb):
    """Custom mutation that respects parameter types."""
    for i in range(len(individual)):
        if random.random() < indpb:
            if i == 0:  # C
                individual[i] = random.choice(C_RANGE)
            elif i == 1:  # penalty
                individual[i] = random.choice(PENALTY_RANGE)
            elif i == 2:  # solver
                individual[i] = random.choice(SOLVER_RANGE)
            elif i == 3:  # max_iter
                individual[i] = random.choice(MAX_ITER_RANGE)
    return individual,

In [ ]:
def create_toolbox(X, y):
    """Create the DEAP toolbox for Logistic Regression."""
    toolbox = base.Toolbox()

    # Define genes
    toolbox.register("C", random.choice, C_RANGE)
    toolbox.register("penalty", random.choice, PENALTY_RANGE)
    toolbox.register("solver", random.choice, SOLVER_RANGE)
    toolbox.register("max_iter", random.choice, MAX_ITER_RANGE)

    # Create individual and population
    toolbox.register("individual", tools.initCycle, creator.Individual,
                     (toolbox.C, toolbox.penalty, toolbox.solver, toolbox.max_iter), n=1)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)

    # Define evaluation function
    def evaluate(individual):
        try:
            lr = LogisticRegression(
                C=individual[0],
                penalty=individual[1],
                solver=individual[2],
                max_iter=individual[3],
                random_state=42
            )

            scores = cross_val_score(lr, X, y, cv=3, scoring='f1_weighted')
            return scores.mean(),
        except Exception as e:
            print(f"Error with individual: {individual}")
            print(f"Error message: {str(e)}")
            return 0.0,

    toolbox.register("evaluate", evaluate)

    # Genetic operators
    toolbox.register("mate", tools.cxTwoPoint)
    toolbox.register("mutate", custom_mutation, indpb=0.2)
    toolbox.register("select", tools.selTournament, tournsize=3)

    return toolbox

In [ ]:
def run_genetic_optimization(X, y, population_size=20, generations=10):
    """Run Genetic Algorithm optimization for Logistic Regression."""
    # Define fitness and individual
    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMax)

    toolbox = create_toolbox(X, y)

    # Create initial population
    population = toolbox.population(n=population_size)

    # Statistics setup
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)

    # Run the algorithm
    final_pop, logbook = algorithms.eaSimple(
        population, toolbox, cxpb=0.7, mutpb=0.2, ngen=generations, stats=stats, verbose=True
    )

    # Get the best solution
    best_solution = tools.selBest(final_pop, k=1)[0]

    best_params = {
        'C': best_solution[0],
        'penalty': best_solution[1],
        'solver': best_solution[2],
        'max_iter': best_solution[3]
    }
    return best_params, logbook

In [ ]:
def optimize_all_targets(X, y_df, population_size=50, generations=5):
    best_params_per_target = {}
    models = {}

    for column in y_df.columns:
        print(f"\nOptimizing for target: {column}")
        best_params, logbook = run_genetic_optimization(X, y_df[column],
                                                      population_size=population_size,
                                                      generations=generations)

        best_params_per_target[column] = best_params

        # Train final model with best parameters
        model = LogisticRegression(**best_params, random_state=42)
        model.fit(X, y_df[column])
        models[column] = model

        print(f"\nBest parameters for {column}:")
        for param, value in best_params.items():
            print(f"{param}: {value}")

    return best_params_per_target, models

In [ ]:
X_processed, y_encoded, label_encoders = prepare_data(
    df["clean_text"],
    y
)

In [ ]:
best_params_per_target, trained_models = optimize_all_targets(X_processed, y_encoded)


Optimizing for target: Acting


/usr/local/lib/python3.11/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.11/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.

gen	nevals	avg     	std      	min     	max     
0  	50    	0.694204	0.0383392	0.646345	0.742165


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which 

1  	36    	0.731553	0.0115606	0.70357 	0.742165


...

1  	33    	0.629922	0.0429227	0.378375	0.651588


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


2  	36    	0.648561	0.00532853	0.625347	0.651588


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


3  	32    	0.649069	0.00617656	0.622564	0.651588
4  	42    	0.649846	0.00689269	0.622564	0.651588
5  	39    	0.643181	0.0437368 	0.347751	0.651588

Best parameters for ovr_sent:
C: 1.0
penalty: l2
solver: saga
max_iter: 200


In [ ]:
print("\nBest parameters found:")
for param, value in best_params_per_target.items():
    print(f"{param}: {value}")


Best parameters found:
Acting: {'C': 10.0, 'penalty': 'l2', 'solver': 'saga', 'max_iter': 200}
direction: {'C': 10.0, 'penalty': 'l2', 'solver': 'liblinear', 'max_iter': 200}
Music: {'C': 10.0, 'penalty': 'l1', 'solver': 'saga', 'max_iter': 500}
Genre: {'C': 1.0, 'penalty': 'l1', 'solver': 'liblinear', 'max_iter': 200}
excitement: {'C': 1.0, 'penalty': 'l2', 'solver': 'saga', 'max_iter': 300}
ovr_sent: {'C': 1.0, 'penalty': 'l2', 'solver': 'saga', 'max_iter': 500}


In [ ]:
def prepare_data(df, aspect_column):
    """Prepare data for a specific aspect"""
    # Encode text to numerical values if needed
    # le = LabelEncoder()
    # Changed to return a DataFrame instead of a Series
    X = df[['clean_text']]
    y = df[aspect_column]
    return X, y

In [ ]:
def train_evaluate_model(X, y, aspect, sampling_strategy='none'):
    """Train and evaluate SVM model with different sampling techniques"""
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = []

    vectorizer = TfidfVectorizer(max_features=5000,
                                ngram_range=(1, 2),
                                min_df=2,
                                max_df=0.95)

    # Transform features and encode labels to categorical
    X_tfidf = vectorizer.fit_transform(X['clean_text'])

    # Encode target labels to categorical for CategoricalNB
    le = LabelEncoder()
    y_encoded = le.fit_transform(y)

    # Print the mapping between original and encoded labels
    print("Label Mapping:")
    for original_label, encoded_label in zip(le.classes_, le.transform(le.classes_)):
        print(f"{original_label} -> {encoded_label}")

    for fold, (train_idx, val_idx) in enumerate(skf.split(X_tfidf, y_encoded)):  # Split using encoded labels
        X_train, X_val = X_tfidf[train_idx], X_tfidf[val_idx]
        y_train, y_val = y_encoded[train_idx], y_encoded[val_idx]  # Use encoded labels

        # Apply sampling strategy
        if sampling_strategy == 'oversample':
            sampler = SMOTE(random_state=42)
            X_train, y_train = sampler.fit_resample(X_train, y_train)
        elif sampling_strategy == 'undersample':
            sampler = RandomUnderSampler(random_state=42)
            X_train, y_train = sampler.fit_resample(X_train, y_train)

        # Convert X_train and X_val to dense arrays
        X_train = X_train.toarray()
        X_val = X_val.toarray()

        if aspect == "acting":
          model =  LogisticRegression(
                  C=10.0,
                  penalty='l2',
                  solver="saga",
                  max_iter=200,
                  random_state=42,
                  class_weight='balanced'
              )
        elif aspect == "direction":
          model =  LogisticRegression(
                  C=10.0,
                  penalty="l2",
                  solver="liblinear",
                  max_iter=200,
                  random_state=42,

                  class_weight='balanced'
              )
        elif aspect == "music":
          model =  LogisticRegression(
                  C=10.0,
                  penalty="l1",
                  solver="saga",
                  max_iter=500,
                  random_state=42,

                  class_weight='balanced'
              )
        elif aspect == "genre":
          model =  LogisticRegression(
                  C=1.0,
                  penalty="l1",
                  solver="liblinear",
                  max_iter=200,
                  random_state=42,

                  class_weight='balanced'
              )
        elif aspect == "excitement":
          model =  LogisticRegression(
                  C=1.0,
                  penalty='l2',
                  solver="saga",
                  max_iter=300,
                  random_state=42,

                  class_weight='balanced'
              )
        elif aspect == "ovr_sent":
          model =  LogisticRegression(
                  C=1.0,
                  penalty="l2",
                  solver="saga",
                  max_iter=500,
                  random_state=42,

                  class_weight='balanced'
              )
        else:
          raise ValueError(f"Unknown aspect: {aspect}")
        model.fit(X_train, y_train)

        # Evaluate
        y_pred = model.predict(X_val)
        score = accuracy_score(y_val, y_pred)  # Use encoded labels for evaluation
        scores.append(score)

        print(f"\nFold {fold + 1} Results:")
        print(classification_report(y_val, y_pred))  # Use encoded labels for classification report

    return np.mean(scores), np.std(scores)

In [ ]:
def analyze_all_aspects_undersample(df, aspects,sampling_techniques):
    """Analyze all aspects with different sampling techniques"""
    results = {}
    # sampling_techniques = ['none', 'oversample', 'undersample']


    for aspect in aspects:
        print(f"\nAnalyzing aspect: {aspect}")
        X, y = prepare_data(df, aspect)

        aspect_results = {}
        for technique in sampling_techniques:
            print(f"\nUsing {technique} sampling:")
            mean_score, std_score = train_evaluate_model(X, y,aspect.lower(),technique)
            aspect_results[technique] = {
                'mean_accuracy': mean_score,
                'std_accuracy': std_score
            }
            print(f"Mean Accuracy: {mean_score:.4f} (±{std_score:.4f})")

        results[aspect] = aspect_results

    return results

In [ ]:
aspects = ['Acting', 'direction', 'Music',  'Genre', 'excitement','ovr_sent']
sampling_techniques_under = ['undersample']
results = analyze_all_aspects_undersample(df, aspects, sampling_techniques_under)


Analyzing aspect: Acting

Using undersample sampling:
Label Mapping:
-1 -> 0
0 -> 1
1 -> 2

Fold 1 Results:
              precision    recall  f1-score   support

           0       0.10      0.62      0.17        34
           1       0.90      0.58      0.70       770
           2       0.47      0.69      0.56       221

    accuracy                           0.60      1025
   macro avg       0.49      0.63      0.48      1025
weighted avg       0.78      0.60      0.65      1025


Fold 2 Results:
              precision    recall  f1-score   support

           0       0.08      0.61      0.14        33
           1       0.88      0.55      0.68       769
           2       0.46      0.58      0.51       222

    accuracy                           0.56      1024
   macro avg       0.47      0.58      0.44      1024
weighted avg       0.76      0.56      0.62      1024


Fold 3 Results:
              precision    recall  f1-score   support

           0       0.09      0.64      0

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(



Fold 1 Results:
              precision    recall  f1-score   support

           0       0.22      0.59      0.32        22
           1       0.96      0.98      0.97       897
           2       0.81      0.42      0.55       106

    accuracy                           0.91      1025
   macro avg       0.66      0.66      0.61      1025
weighted avg       0.93      0.91      0.91      1025



/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(



Fold 2 Results:
              precision    recall  f1-score   support

           0       0.18      0.71      0.29        21
           1       0.96      0.92      0.94       897
           2       0.62      0.52      0.57       106

    accuracy                           0.87      1024
   macro avg       0.59      0.72      0.60      1024
weighted avg       0.91      0.87      0.89      1024



...


Fold 4 Results:
              precision    recall  f1-score   support

           0       0.20      0.62      0.31        21
           1       0.97      0.92      0.95       897
           2       0.64      0.62      0.63       106

    accuracy                           0.89      1024
   macro avg       0.60      0.72      0.63      1024
weighted avg       0.92      0.89      0.90      1024



/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(



Fold 5 Results:
              precision    recall  f1-score   support

           0       0.19      0.64      0.30        22
           1       0.96      0.95      0.96       896
           2       0.66      0.44      0.53       106

    accuracy                           0.89      1024
   macro avg       0.61      0.68      0.59      1024
weighted avg       0.92      0.89      0.90      1024

Mean Accuracy: 0.8889 (±0.0119)

Analyzing aspect: Genre

Using undersample sampling:
Label Mapping:
-1 -> 0
0 -> 1
1 -> 2

Fold 1 Results:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        10
           1       0.96      0.88      0.92       975
           2       0.50      0.25      0.33        40

    accuracy                           0.84      1025
   macro avg       0.49      0.38      0.42      1025
weighted avg       0.94      0.84      0.89      1025


Fold 2 Results:
              precision    recall  f1-score   support

         

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Label Mapping:
-1 -> 0
0 -> 1
1 -> 2

Fold 1 Results:
              precision    recall  f1-score   support

           0       0.38      0.68      0.49       152
           1       0.75      0.59      0.66       583
           2       0.59      0.60      0.60       290

    accuracy                           0.61      1025
   macro avg       0.57      0.62      0.58      1025
weighted avg       0.65      0.61      0.62      1025


Fold 2 Results:
              precision    recall  f1-score   support

           0       0.39      0.74      0.51       152
           1       0.78      0.58      0.66       583
           2       0.63      0.66      0.65       289

    accuracy                           0.62      1024
   macro avg       0.60      0.66      0.61      1024
weighted avg       0.68      0.62      0.64      1024


Fold 3 Results:
              precision    recall  f1-score   support

           0       0.37      0.72      0.49       151
           1       0.75      0.56      0.

In [ ]:
sampling_techniques_over = ['oversample']
results = analyze_all_aspects_undersample(df, aspects, sampling_techniques_over)


Analyzing aspect: Acting

Using oversample sampling:
Label Mapping:
-1 -> 0
0 -> 1
1 -> 2

Fold 1 Results:
              precision    recall  f1-score   support

           0       0.14      0.21      0.17        34
           1       0.90      0.85      0.87       770
           2       0.61      0.69      0.65       221

    accuracy                           0.79      1025
   macro avg       0.55      0.58      0.56      1025
weighted avg       0.81      0.79      0.80      1025


Fold 2 Results:
              precision    recall  f1-score   support

           0       0.11      0.15      0.13        33
           1       0.89      0.83      0.86       769
           2       0.57      0.68      0.62       222

    accuracy                           0.77      1024
   macro avg       0.52      0.55      0.54      1024
weighted avg       0.79      0.77      0.78      1024


Fold 3 Results:
              precision    recall  f1-score   support

           0       0.16      0.24      0.

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(



Fold 1 Results:
              precision    recall  f1-score   support

           0       0.45      0.23      0.30        22
           1       0.98      0.97      0.98       897
           2       0.71      0.84      0.77       106

    accuracy                           0.94      1025
   macro avg       0.72      0.68      0.68      1025
weighted avg       0.94      0.94      0.94      1025



/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(



Fold 2 Results:
              precision    recall  f1-score   support

           0       0.56      0.43      0.49        21
           1       0.98      0.92      0.95       897
           2       0.53      0.80      0.64       106

    accuracy                           0.90      1024
   macro avg       0.69      0.72      0.69      1024
weighted avg       0.92      0.90      0.91      1024



...


Fold 3 Results:
              precision    recall  f1-score   support

           0       0.60      0.43      0.50        21
           1       0.97      0.94      0.95       897
           2       0.58      0.78      0.66       106

    accuracy                           0.91      1024
   macro avg       0.72      0.72      0.71      1024
weighted avg       0.92      0.91      0.92      1024



/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(



Fold 4 Results:
              precision    recall  f1-score   support

           0       0.50      0.33      0.40        21
           1       0.98      0.98      0.98       897
           2       0.80      0.82      0.81       106

    accuracy                           0.95      1024
   macro avg       0.76      0.71      0.73      1024
weighted avg       0.95      0.95      0.95      1024



/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(



Fold 5 Results:
              precision    recall  f1-score   support

           0       0.42      0.23      0.29        22
           1       0.98      0.93      0.95       896
           2       0.55      0.80      0.65       106

    accuracy                           0.90      1024
   macro avg       0.65      0.65      0.63      1024
weighted avg       0.92      0.90      0.91      1024

Mean Accuracy: 0.9219 (±0.0204)

Analyzing aspect: Genre

Using oversample sampling:
Label Mapping:
-1 -> 0
0 -> 1
1 -> 2

Fold 1 Results:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        10
           1       0.97      0.97      0.97       975
           2       0.40      0.42      0.41        40

    accuracy                           0.94      1025
   macro avg       0.46      0.46      0.46      1025
weighted avg       0.94      0.94      0.94      1025


Fold 2 Results:
              precision    recall  f1-score   support

          

In [ ]:
import re

def custom_tokenizer(text):
    # Split by spaces and punctuation, retain Tamil/English words
    tokens = re.findall(r"[\w']+|[!\"#$%&'()*+,-./:;<=>?@[\]^_`{|}~]", text)
    return tokens

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Use custom tokenizer and include n-grams (e.g., unigrams + bigrams)
tfidf = TfidfVectorizer(
    tokenizer=custom_tokenizer,
    ngram_range=(1, 2),  # Adjust based on performance
    max_features=5000    # Prevent overfitting
)
X = tfidf.fit_transform(df['clean_text'])

/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
aspect_columns = ['Acting', 'direction', 'Music',  'Genre', 'excitement','ovr_sent']

y = df[aspect_columns]

In [ ]:
from sklearn.model_selection import train_test_split

# Assuming 'X' and 'y' are your feature and target data respectively
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42  # Adjust test_size as needed
)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier

# Handle class imbalance with class weights
model1 = MultiOutputClassifier(
    LogisticRegression(
        multi_class='multinomial',  # For 3 classes (-1, 0, 1)
        solver='lbfgs',
        class_weight='balanced'     # Adjust for imbalanced labels
    )
)

# Train on all aspect labels (acting, direction, music, etc.)

model1.fit(X_train, y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and wi

MultiOutputClassifier(estimator=LogisticRegression(class_weight='balanced',
                                                   multi_class='multinomial'))

In [ ]:
from sklearn.metrics import classification_report, hamming_loss

y_pred = model1.predict(X_test)
for i, target_name in enumerate(aspect_columns):
    print(f"Classification Report for {target_name}:\n")
    print(classification_report(y_test.iloc[:, i], y_pred[:, i]))
    print("\n")
individual_hamming_losses = []
for i in range(y_test.shape[1]):  # Iterate through aspects
    individual_hamming_losses.append(hamming_loss(y_test.iloc[:, i], y_pred[:, i]))

# Calculate average Hamming Loss
average_hamming_loss = np.mean(individual_hamming_losses)

print("Average Hamming Loss:", average_hamming_loss)

Classification Report for Acting:

              precision    recall  f1-score   support

          -1       0.27      0.27      0.27        41
           0       0.89      0.91      0.90       946
           1       0.71      0.66      0.68       294

    accuracy                           0.83      1281
   macro avg       0.62      0.61      0.62      1281
weighted avg       0.83      0.83      0.83      1281



Classification Report for direction:

              precision    recall  f1-score   support

          -1       0.42      0.58      0.49       173
           0       0.91      0.83      0.87      1034
           1       0.45      0.58      0.51        74

    accuracy                           0.78      1281
   macro avg       0.59      0.66      0.62      1281
weighted avg       0.82      0.78      0.80      1281



Classification Report for Music:

              precision    recall  f1-score   support

          -1       0.44      0.35      0.39        23
           0      

In [ ]:
y_test

,Acting,direction,Music,Genre,excitement,ovr_sent
530,0,0,0,-1,0,-1
1729,0,0,0,0,0,-1
1010,0,0,0,0,1,1
1149,0,-1,0,0,0,-1
1383,-1,0,0,0,0,-1
...,...,...,...,...,...,...
3049,0,-1,0,0,0,-1
3614,0,0,0,0,1,1
73,0,0,0,0,0,-1
1909,0,0,0,0,-1,-1


In [ ]:
y_pred

array([[ 0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0, -1],
       [ 0,  0,  0,  0,  1,  1],
       ...,
       [ 0, -1,  0,  0, -1, -1],
       [ 0,  0,  0,  0,  0,  1],
       [ 0, -1,  0,  0,  0, -1]])

In [ ]:
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(tokenizer=custom_tokenizer)),
    ('clf', MultiOutputClassifier(
        LogisticRegression(class_weight='balanced', multi_class='multinomial')
    ))
])

# Train
pipeline.fit(df['clean_text'], df[aspect_columns])

In [ ]:
def train_final_models_grid_search(df, aspects):
    """
    For each aspect, perform a grid search to find the best SVC parameters using a pipeline.
    Uses an 80/20 train-test split, prints training and test accuracy and classification reports,
    and returns the best models along with their label encoders.
    """
    best_models = {}
    train_reports = {}
    test_reports = {}

    # Define a parameter grid for the SVC inside a pipeline
    param_grid = {
      'clf__C': [0.01, 0.1, 1, 10, 100],  # Regularization strength
      'clf__penalty': ['l1', 'l2'],  # Regularization type
      'clf__solver': ['liblinear', 'saga'],  # Algorithm to use
      'clf__max_iter': [100, 200, 500]  # Maximum iterations
}

    for aspect in aspects:
        print(f"\n=== Processing aspect: {aspect} ===")
        X, y = prepare_data(df, aspect)
        # Encode target labels to numerical values
        le = LabelEncoder()
        y_encoded = le.fit_transform(y)

        # Split data into training and testing sets (80/20 split)
        X_train, X_test, y_train, y_test = train_test_split(
            X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
        )

        # Create a pipeline: first vectorize text, then classify with SVC
        pipeline = Pipeline([
            ('tfidf', TfidfVectorizer(max_features=5000,
                                      ngram_range=(1, 2),
                                      min_df=2,
                                      max_df=0.95)),
            ('clf', LogisticRegression(random_state=42,class_weight='balanced', multi_class='multinomial'))
        ])

        # Set up GridSearchCV with 5-fold cross-validation
        cv_strategy = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        grid = GridSearchCV(pipeline, param_grid, cv=cv_strategy, scoring='accuracy', n_jobs=-1)
        grid.fit(X_train['clean_text'], y_train)

        print(f"Best parameters for aspect '{aspect}': {grid.best_params_}")

        best_estimator = grid.best_estimator_

        # Evaluate on training data
        y_train_pred = best_estimator.predict(X_train['clean_text'])
        train_acc = accuracy_score(y_train, y_train_pred)
        print("Training Accuracy: {:.4f}".format(train_acc))
        train_report = classification_report(y_train, y_train_pred)
        print("Training Classification Report:\n", train_report)

        # Evaluate on test data
        y_test_pred = best_estimator.predict(X_test['clean_text'])
        test_acc = accuracy_score(y_test, y_test_pred)
        print("Test Accuracy: {:.4f}".format(test_acc))
        test_report = classification_report(y_test, y_test_pred)
        print("Test Classification Report:\n", test_report)

        best_models[aspect.lower()] = {
            "model": best_estimator,
            "label_encoder": le
        }
        train_reports[aspect.lower()] = train_report
        test_reports[aspect.lower()] = test_report

    return best_models, train_reports, test_reports

In [ ]:
def predict_all_aspects(sentence, final_models):
    """
    Given a sentence, predict its sentiment for all aspects using the final trained models.
    Returns a dictionary with aspect names as keys and predicted labels as values.
    """
    predictions = {}
    for aspect, model_info in final_models.items():
        model = model_info['model']
        le = model_info['label_encoder']
        # The pipeline expects raw text
        pred_encoded = model.predict([sentence])[0]
        pred_label = le.inverse_transform([pred_encoded])[0]
        predictions[aspect] = pred_label
    return predictions

In [ ]:
aspects = ['Acting', 'direction', 'Music',  'Genre', 'excitement','ovr_sent']

In [ ]:
best_models, train_reports, test_reports = train_final_models_grid_search(df, aspects)


=== Processing aspect: Acting ===


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
150 fits failed out of a total of 300.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
150 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py", line 662, in fit
    self._final_estimator.fit(Xt, y, *

Best parameters for aspect 'Acting': {'clf__C': 10, 'clf__max_iter': 200, 'clf__penalty': 'l1', 'clf__solver': 'saga'}
Training Accuracy: 0.9406
Training Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.99      0.91       132
           1       0.94      0.99      0.96      2966
           2       0.99      0.76      0.86       843

    accuracy                           0.94      3941
   macro avg       0.92      0.91      0.91      3941
weighted avg       0.94      0.94      0.94      3941

Test Accuracy: 0.8225
Test Classification Report:
               precision    recall  f1-score   support

           0       0.18      0.24      0.21        33
           1       0.87      0.93      0.90       742
           2       0.77      0.53      0.62       211

    accuracy                           0.82       986
   macro avg       0.61      0.57      0.58       986
weighted avg       0.82      0.82      0.82       986


=== Proc

/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
150 fits failed out of a total of 300.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
150 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py", line 662, in fit
    self._final_estimator.fit(Xt, y, *

Best parameters for aspect 'direction': {'clf__C': 100, 'clf__max_iter': 500, 'clf__penalty': 'l2', 'clf__solver': 'saga'}
Training Accuracy: 0.9822
Training Classification Report:
               precision    recall  f1-score   support

           0       0.93      1.00      0.96       554
           1       1.00      0.98      0.99      3145
           2       0.91      0.99      0.95       242

    accuracy                           0.98      3941
   macro avg       0.94      0.99      0.97      3941
weighted avg       0.98      0.98      0.98      3941

Test Accuracy: 0.7566
Test Classification Report:
               precision    recall  f1-score   support

           0       0.36      0.50      0.42       138
           1       0.90      0.83      0.86       787
           2       0.34      0.39      0.37        61

    accuracy                           0.76       986
   macro avg       0.53      0.57      0.55       986
weighted avg       0.79      0.76      0.77       986


=== 

...

Best parameters for aspect 'Genre': {'clf__C': 100, 'clf__max_iter': 200, 'clf__penalty': 'l2', 'clf__solver': 'saga'}
Training Accuracy: 0.9736
Training Classification Report:
               precision    recall  f1-score   support

           0       0.40      1.00      0.57        42
           1       1.00      0.97      0.99      3742
           2       0.79      0.99      0.88       157

    accuracy                           0.97      3941
   macro avg       0.73      0.99      0.81      3941
weighted avg       0.99      0.97      0.98      3941

Test Accuracy: 0.8773
Test Classification Report:
               precision    recall  f1-score   support

           0       0.02      0.09      0.03        11
           1       0.97      0.90      0.94       936
           2       0.33      0.46      0.38        39

    accuracy                           0.88       986
   macro avg       0.44      0.49      0.45       986
weighted avg       0.93      0.88      0.90       986


=== Proc

/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
150 fits failed out of a total of 300.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
150 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py", line 662, in fit
    self._final_estimator.fit(Xt, y, *

Best parameters for aspect 'excitement': {'clf__C': 1, 'clf__max_iter': 100, 'clf__penalty': 'l2', 'clf__solver': 'saga'}
Training Accuracy: 0.8447
Training Classification Report:
               precision    recall  f1-score   support

           0       0.68      0.97      0.80       594
           1       0.94      0.80      0.87      2244
           2       0.81      0.86      0.83      1103

    accuracy                           0.84      3941
   macro avg       0.81      0.88      0.83      3941
weighted avg       0.86      0.84      0.85      3941

Test Accuracy: 0.6531
Test Classification Report:
               precision    recall  f1-score   support

           0       0.44      0.70      0.54       148
           1       0.77      0.63      0.69       562
           2       0.64      0.68      0.66       276

    accuracy                           0.65       986
   macro avg       0.62      0.67      0.63       986
weighted avg       0.68      0.65      0.66       986


=== P

/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
150 fits failed out of a total of 300.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
150 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py", line 662, in fit
    self._final_estimator.fit(Xt, y, *

Best parameters for aspect 'ovr_sent': {'clf__C': 1, 'clf__max_iter': 100, 'clf__penalty': 'l2', 'clf__solver': 'saga'}
Training Accuracy: 0.8721
Training Classification Report:
               precision    recall  f1-score   support

           0       0.86      0.90      0.88      1355
           1       0.68      0.92      0.78       565
           2       0.96      0.84      0.90      2021

    accuracy                           0.87      3941
   macro avg       0.84      0.89      0.85      3941
weighted avg       0.89      0.87      0.88      3941

Test Accuracy: 0.6907
Test Classification Report:
               precision    recall  f1-score   support

           0       0.68      0.76      0.71       339
           1       0.36      0.45      0.40       141
           2       0.84      0.71      0.77       506

    accuracy                           0.69       986
   macro avg       0.62      0.64      0.63       986
weighted avg       0.72      0.69      0.70       986



In [ ]:
sample_sentence = "superster nalla illa "
predictions = predict_all_aspects(sample_sentence, best_models)

print("\nPredictions for the sample sentence:")
for aspect, pred in predictions.items():
    print(f"{aspect.capitalize()}: {pred}")


Predictions for the sample sentence:
Acting: 0
Direction: 0
Music: 0
Genre: 0
Excitement: 0
Ovr_sent: -1


In [ ]:
def train_final_models_grid_search(df, aspects):
    """
    For each aspect, perform a grid search to find the best SVC parameters using a pipeline.
    Uses an 80/20 train-test split, prints training and test accuracy and classification reports,
    and returns the best models along with their label encoders.
    """
    best_models = {}
    train_reports = {}
    test_reports = {}

    # Define a parameter grid for the SVC inside a pipeline
    param_grid = {
    'tfidf__max_features': [1000, 5000, 10000], # Example values
    'tfidf__ngram_range': [(1, 1), (1, 2)],     # Example values
    'clf__C': [0.01, 0.1, 1, 10, 100],
    'clf__penalty': ['l1', 'l2'],
    'clf__solver': ['liblinear', 'saga'],
    'clf__max_iter': [100, 200, 500]
}

    for aspect in aspects:
        print(f"\n=== Processing aspect: {aspect} ===")
        X, y = prepare_data(df, aspect)
        # Encode target labels to numerical values
        le = LabelEncoder()
        y_encoded = le.fit_transform(y)

        # Split data into training and testing sets (80/20 split)
        X_train, X_test, y_train, y_test = train_test_split(
            X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
        )


        # Create a pipeline: first vectorize text, then classify with SVC
        pipeline = Pipeline([
            ('tfidf', TfidfVectorizer(min_df=2,
                                      max_df=0.95)),
            ('clf', LogisticRegression(random_state=42,class_weight='balanced', multi_class='multinomial'))
        ])

        # Set up GridSearchCV with 5-fold cross-validation
        cv_strategy = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        grid = GridSearchCV(pipeline, param_grid, cv=cv_strategy, scoring='f1_macro', n_jobs=-1)
        grid.fit(X_train['clean_text'], y_train)

        print(f"Best parameters for aspect '{aspect}': {grid.best_params_}")

        best_estimator = grid.best_estimator_

        # Evaluate on training data
        y_train_pred = best_estimator.predict(X_train['clean_text'])
        train_acc = accuracy_score(y_train, y_train_pred)
        print("Training Accuracy: {:.4f}".format(train_acc))
        train_report = classification_report(y_train, y_train_pred)
        print("Training Classification Report:\n", train_report)

        # Evaluate on test data
        y_test_pred = best_estimator.predict(X_test['clean_text'])
        test_acc = accuracy_score(y_test, y_test_pred)
        print("Test Accuracy: {:.4f}".format(test_acc))
        test_report = classification_report(y_test, y_test_pred)
        print("Test Classification Report:\n", test_report)

        best_models[aspect.lower()] = {
            "model": best_estimator,
            "label_encoder": le
        }
        train_reports[aspect.lower()] = train_report
        test_reports[aspect.lower()] = test_report

    return best_models, train_reports, test_reports

In [ ]:
best_models, train_reports, test_reports = train_final_models_grid_search(df, aspects)


=== Processing aspect: Acting ===


/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
900 fits failed out of a total of 1800.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
900 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py", line 662, in fit
    self._final_estimator.fit(Xt, y, 

Best parameters for aspect 'Acting': {'clf__C': 10, 'clf__max_iter': 200, 'clf__penalty': 'l1', 'clf__solver': 'saga', 'tfidf__max_features': 5000, 'tfidf__ngram_range': (1, 2)}
Training Accuracy: 0.9406
Training Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.99      0.91       132
           1       0.94      0.99      0.96      2966
           2       0.99      0.76      0.86       843

    accuracy                           0.94      3941
   macro avg       0.92      0.91      0.91      3941
weighted avg       0.94      0.94      0.94      3941

Test Accuracy: 0.8225
Test Classification Report:
               precision    recall  f1-score   support

           0       0.18      0.24      0.21        33
           1       0.87      0.93      0.90       742
           2       0.77      0.53      0.62       211

    accuracy                           0.82       986
   macro avg       0.61      0.57      0.58       986
weigh

/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
900 fits failed out of a total of 1800.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
900 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py", line 662, in fit
    self._final_estimator.fit(Xt, y, 

Best parameters for aspect 'direction': {'clf__C': 10, 'clf__max_iter': 100, 'clf__penalty': 'l1', 'clf__solver': 'saga', 'tfidf__max_features': 5000, 'tfidf__ngram_range': (1, 2)}
Training Accuracy: 0.9472
Training Classification Report:
               precision    recall  f1-score   support

           0       0.77      1.00      0.87       554
           1       1.00      0.93      0.97      3145
           2       0.87      1.00      0.93       242

    accuracy                           0.95      3941
   macro avg       0.88      0.98      0.92      3941
weighted avg       0.96      0.95      0.95      3941

Test Accuracy: 0.7525
Test Classification Report:
               precision    recall  f1-score   support

           0       0.35      0.51      0.41       138
           1       0.91      0.82      0.86       787
           2       0.38      0.46      0.41        61

    accuracy                           0.75       986
   macro avg       0.54      0.59      0.56       986
we

...

Best parameters for aspect 'Genre': {'clf__C': 100, 'clf__max_iter': 100, 'clf__penalty': 'l2', 'clf__solver': 'saga', 'tfidf__max_features': 5000, 'tfidf__ngram_range': (1, 2)}
Training Accuracy: 0.9515
Training Classification Report:
               precision    recall  f1-score   support

           0       0.30      1.00      0.46        42
           1       1.00      0.95      0.97      3742
           2       0.62      0.99      0.77       157

    accuracy                           0.95      3941
   macro avg       0.64      0.98      0.73      3941
weighted avg       0.98      0.95      0.96      3941

Test Accuracy: 0.8529
Test Classification Report:
               precision    recall  f1-score   support

           0       0.02      0.09      0.03        11
           1       0.98      0.87      0.92       936
           2       0.25      0.59      0.35        39

    accuracy                           0.85       986
   macro avg       0.41      0.52      0.43       986
weigh

/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
900 fits failed out of a total of 1800.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
900 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py", line 662, in fit
    self._final_estimator.fit(Xt, y, 

Best parameters for aspect 'excitement': {'clf__C': 1, 'clf__max_iter': 100, 'clf__penalty': 'l2', 'clf__solver': 'saga', 'tfidf__max_features': 5000, 'tfidf__ngram_range': (1, 2)}
Training Accuracy: 0.8447
Training Classification Report:
               precision    recall  f1-score   support

           0       0.68      0.97      0.80       594
           1       0.94      0.80      0.87      2244
           2       0.81      0.86      0.83      1103

    accuracy                           0.84      3941
   macro avg       0.81      0.88      0.83      3941
weighted avg       0.86      0.84      0.85      3941

Test Accuracy: 0.6531
Test Classification Report:
               precision    recall  f1-score   support

           0       0.44      0.70      0.54       148
           1       0.77      0.63      0.69       562
           2       0.64      0.68      0.66       276

    accuracy                           0.65       986
   macro avg       0.62      0.67      0.63       986
we

/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py:528: FitFailedWarning: 
900 fits failed out of a total of 1800.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
900 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.11/dist-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py", line 662, in fit
    self._final_estimator.fit(Xt, y, 

Best parameters for aspect 'ovr_sent': {'clf__C': 1, 'clf__max_iter': 100, 'clf__penalty': 'l2', 'clf__solver': 'saga', 'tfidf__max_features': 5000, 'tfidf__ngram_range': (1, 2)}
Training Accuracy: 0.8721
Training Classification Report:
               precision    recall  f1-score   support

           0       0.86      0.90      0.88      1355
           1       0.68      0.92      0.78       565
           2       0.96      0.84      0.90      2021

    accuracy                           0.87      3941
   macro avg       0.84      0.89      0.85      3941
weighted avg       0.89      0.87      0.88      3941

Test Accuracy: 0.6907
Test Classification Report:
               precision    recall  f1-score   support

           0       0.68      0.76      0.71       339
           1       0.36      0.45      0.40       141
           2       0.84      0.71      0.77       506

    accuracy                           0.69       986
   macro avg       0.62      0.64      0.63       986
weig

In [ ]:
sample_sentence = "pei padam maari theriyuthu  "
predictions = predict_all_aspects(sample_sentence, best_models)

print("\nPredictions for the sample sentence:")
for aspect, pred in predictions.items():
    print(f"{aspect.capitalize()}: {pred}")


Predictions for the sample sentence:
Acting: 0
Direction: 0
Music: 0
Genre: 0
Excitement: -1
Ovr_sent: -1


In [ ]:
def custom_tokenizer(text):
    return text.split()

In [ ]:
train_df = pd.read_csv("train_data_greater than 5 word.csv")
test_df = pd.read_csv("test_data_greater than 5 words.csv")

In [ ]:
def train_final_models_grid_search(train_df, test_df, aspects):
    """
    For each aspect:
    1. Perform grid search on the training data to find the best parameters.
    2. Using the best parameters, perform 10-fold cross-validation,
       returning both the average accuracy and the list of fold estimators.
    3. Select the best estimator from CV (highest test score) and evaluate it on test data.
    Returns the best models along with their label encoders and classification reports.
    """
    best_models = {}
    train_reports = {}
    test_reports = {}

    # Parameter grid for the classifier
    param_grid = {
    'clf__C': [0.01, 0.1, 1, 10,100],
    'clf__penalty': ['l1', 'l2'],
    'clf__solver': ['liblinear', 'saga'],
    'clf__max_iter': [100, 200, 500]
}

    for aspect in aspects:
        print(f"\n=== Processing aspect: {aspect} ===")
        # Prepare data for current aspect
        X_train, y_train = prepare_data(train_df, aspect)
        X_test, y_test = prepare_data(test_df, aspect)

        # Encode target labels to numerical values
        le = LabelEncoder()
        y_train_encoded = le.fit_transform(y_train)
        y_test_encoded = le.fit_transform(y_test)

        # Build initial pipeline with a placeholder classifier
        pipeline = Pipeline([
            ('tfidf', TfidfVectorizer(max_features=5000,
                                      ngram_range=(1, 2),
                                      min_df=5,
                                      max_df=0.90,
                                      tokenizer=custom_tokenizer)),
            ('clf', LogisticRegression(random_state=42, class_weight='balanced'))
        ])

        # 10-fold CV strategy for grid search
        cv_strategy = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

        # Ensure there are no missing or empty texts in training set
        X_train = X_train.dropna(subset=['clean_text'])
        X_train = X_train[X_train['clean_text'].str.strip() != '']

        # --- Step 1: Grid Search ---
        grid = GridSearchCV(pipeline, param_grid, cv=cv_strategy,
                            scoring='balanced_accuracy', n_jobs=-1, return_train_score=True)
        grid.fit(X_train['clean_text'], y_train_encoded)
        best_params = grid.best_params_
        print(f"Best parameters for aspect '{aspect}': {best_params}")

        # --- Step 2: Build a new pipeline using the best parameters ---
        best_pipeline = Pipeline([
            ('tfidf', TfidfVectorizer(max_features=5000,
                                      ngram_range=(1, 2),
                                      min_df=5,
                                      max_df=0.90,
                                      tokenizer=custom_tokenizer)),
            ('clf', LogisticRegression(random_state=42,
                                             class_weight='balanced',
                                             C=best_params['clf__C'],
                                             penalty=best_params['clf__penalty'],
                                             solver=best_params['clf__solver'],
                                             max_iter=best_params['clf__max_iter']
                                             ))
        ])

        # --- Step 3: Perform 10-Fold Cross Validation with estimator return ---
        cv_results = cross_validate(best_pipeline, X_train['clean_text'], y_train_encoded,
                                    cv=StratifiedKFold(n_splits=10, shuffle=True, random_state=42),
                                    scoring='balanced_accuracy', return_train_score=True,return_estimator=True, n_jobs=-1)
        avg_cv_score_test = cv_results['test_score'].mean()
        avg_cv_score_train = cv_results['train_score'].mean()
        print(avg_cv_score_test)
        print(avg_cv_score_train)


        print(f"Average 10-fold CV test Accuracy for aspect '{aspect}': {avg_cv_score_test:.4f}")
        print(f"Average 10-fold CV train Accuracy for aspect '{aspect}': {avg_cv_score_train:.4f}")

        # --- Step 4: Select the best estimator from the CV folds ---
        best_index = np.argmax(cv_results['test_score'])
        best_cv_estimator = cv_results['estimator'][best_index]

        # Evaluate the selected estimator on the training data (optional)
        y_train_pred = best_cv_estimator.predict(X_train['clean_text'])
        train_acc = accuracy_score(y_train_encoded, y_train_pred)
        print("Final Training Accuracy: {:.4f}".format(train_acc))
        train_report = classification_report(y_train_encoded, y_train_pred)
        print("Final Training Classification Report:\n", train_report)

        # --- Step 5: Evaluate on Test Data ---
        y_test_pred = best_cv_estimator.predict(X_test['clean_text'])
        test_acc = accuracy_score(y_test_encoded, y_test_pred)
        print("Test Accuracy: {:.4f}".format(test_acc))
        test_report = classification_report(y_test_encoded, y_test_pred)
        print("Test Classification Report:\n", test_report)

        # Save results for the current aspect
        best_models[aspect.lower()] = {
            "model": best_cv_estimator,
            "label_encoder": le
        }
        train_reports[aspect.lower()] = train_report
        test_reports[aspect.lower()] = test_report

    return best_models, train_reports, test_reports


In [ ]:
aspects = ['Acting', 'direction', 'Music','ovr_sent']

In [ ]:
best_models, train_reports, test_reports = train_final_models_grid_search(train_df,test_df, aspects)


=== Processing aspect: Acting ===


/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Best parameters for aspect 'Acting': {'clf__C': 0.01, 'clf__max_iter': 100, 'clf__penalty': 'l2', 'clf__solver': 'saga'}
0.6104513723141174
0.7873929200548041
Average 10-fold CV test Accuracy for aspect 'Acting': 0.6105
Average 10-fold CV train Accuracy for aspect 'Acting': 0.7874
Final Training Accuracy: 0.7602
Final Training Classification Report:
               precision    recall  f1-score   support

           0       0.21      0.85      0.34       131
           1       0.92      0.78      0.84      2962
           2       0.63      0.69      0.66       848

    accuracy                           0.76      3941
   macro avg       0.59      0.77      0.61      3941
weighted avg       0.84      0.76      0.79      3941

Test Accuracy: 0.7363
Test Classification Report:
               precision    recall  f1-score   support

           0       0.11      0.35      0.17        34
           1       0.89      0.78      0.83       746
           2       0.59      0.65      0.62       20

/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Best parameters for aspect 'direction': {'clf__C': 0.01, 'clf__max_iter': 100, 'clf__penalty': 'l2', 'clf__solver': 'saga'}
0.638262948371995
0.7236238298468942
Average 10-fold CV test Accuracy for aspect 'direction': 0.6383
Average 10-fold CV train Accuracy for aspect 'direction': 0.7236
Final Training Accuracy: 0.7402
Final Training Classification Report:
               precision    recall  f1-score   support

           0       0.40      0.76      0.52       568
           1       0.95      0.74      0.83      3128
           2       0.41      0.65      0.50       245

    accuracy                           0.74      3941
   macro avg       0.58      0.72      0.62      3941
weighted avg       0.83      0.74      0.77      3941

Test Accuracy: 0.6805
Test Classification Report:
               precision    recall  f1-score   support

           0       0.28      0.61      0.39       124
           1       0.92      0.70      0.79       804
           2       0.30      0.53      0.39 

/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Best parameters for aspect 'Music': {'clf__C': 10, 'clf__max_iter': 200, 'clf__penalty': 'l1', 'clf__solver': 'saga'}
0.6898242227282401
0.9505244504492524
Average 10-fold CV test Accuracy for aspect 'Music': 0.6898
Average 10-fold CV train Accuracy for aspect 'Music': 0.9505
Final Training Accuracy: 0.9328
Final Training Classification Report:
               precision    recall  f1-score   support

           0       0.39      0.97      0.55        89
           1       1.00      0.93      0.96      3446
           2       0.76      0.92      0.84       406

    accuracy                           0.93      3941
   macro avg       0.72      0.94      0.78      3941
weighted avg       0.96      0.93      0.94      3941

Test Accuracy: 0.8824
Test Classification Report:
               precision    recall  f1-score   support

           0       0.15      0.44      0.22        18
           1       0.99      0.90      0.94       867
           2       0.59      0.84      0.69       101

  

/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Best parameters for aspect 'ovr_sent': {'clf__C': 1, 'clf__max_iter': 100, 'clf__penalty': 'l1', 'clf__solver': 'liblinear'}
0.6354442267899455
0.7348654535736401
Average 10-fold CV test Accuracy for aspect 'ovr_sent': 0.6354
Average 10-fold CV train Accuracy for aspect 'ovr_sent': 0.7349
Final Training Accuracy: 0.7452
Final Training Classification Report:
               precision    recall  f1-score   support

           0       0.74      0.78      0.76      1346
           1       0.45      0.64      0.53       567
           2       0.89      0.75      0.82      2028

    accuracy                           0.75      3941
   macro avg       0.69      0.72      0.70      3941
weighted avg       0.78      0.75      0.75      3941

Test Accuracy: 0.6582
Test Classification Report:
               precision    recall  f1-score   support

           0       0.68      0.67      0.67       348
           1       0.32      0.51      0.39       139
           2       0.82      0.69      0.75 

sentence over or equal to 5 words

In [ ]:
best_models, train_reports, test_reports = train_final_models_grid_search(train_df,test_df, aspects)


=== Processing aspect: Acting ===


/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Best parameters for aspect 'Acting': {'clf__C': 0.01, 'clf__max_iter': 100, 'clf__penalty': 'l2', 'clf__solver': 'saga'}
0.6022741707186494
0.8014176563903804
Average 10-fold CV test Accuracy for aspect 'Acting': 0.6023
Average 10-fold CV train Accuracy for aspect 'Acting': 0.8014
Final Training Accuracy: 0.7665
Final Training Classification Report:
               precision    recall  f1-score   support

           0       0.27      0.86      0.41        93
           1       0.93      0.77      0.84      1506
           2       0.65      0.73      0.69       422

    accuracy                           0.77      2021
   macro avg       0.62      0.79      0.65      2021
weighted avg       0.84      0.77      0.79      2021

Test Accuracy: 0.6949
Test Classification Report:
               precision    recall  f1-score   support

           0       0.13      0.31      0.19        26
           1       0.88      0.74      0.80       376
           2       0.49      0.61      0.54        9

/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Best parameters for aspect 'direction': {'clf__C': 0.1, 'clf__max_iter': 100, 'clf__penalty': 'l2', 'clf__solver': 'saga'}
0.640651549473859
0.7872684561094082
Average 10-fold CV test Accuracy for aspect 'direction': 0.6407
Average 10-fold CV train Accuracy for aspect 'direction': 0.7873
Final Training Accuracy: 0.7625
Final Training Classification Report:
               precision    recall  f1-score   support

           0       0.56      0.78      0.65       416
           1       0.93      0.75      0.83      1455
           2       0.45      0.79      0.57       150

    accuracy                           0.76      2021
   macro avg       0.65      0.78      0.69      2021
weighted avg       0.82      0.76      0.78      2021

Test Accuracy: 0.6505
Test Classification Report:
               precision    recall  f1-score   support

           0       0.37      0.62      0.46        87
           1       0.87      0.68      0.77       369
           2       0.27      0.41      0.33  

/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Best parameters for aspect 'Music': {'clf__C': 10, 'clf__max_iter': 100, 'clf__penalty': 'l1', 'clf__solver': 'saga'}
0.6622628252973081
0.9691406543697708
Average 10-fold CV test Accuracy for aspect 'Music': 0.6623
Average 10-fold CV train Accuracy for aspect 'Music': 0.9691
Final Training Accuracy: 0.9673
Final Training Classification Report:
               precision    recall  f1-score   support

           0       0.67      0.95      0.79        63
           1       0.99      0.98      0.98      1743
           2       0.93      0.89      0.91       215

    accuracy                           0.97      2021
   macro avg       0.86      0.94      0.89      2021
weighted avg       0.97      0.97      0.97      2021

Test Accuracy: 0.9091
Test Classification Report:
               precision    recall  f1-score   support

           0       0.28      0.38      0.32        13
           1       0.97      0.94      0.96       428
           2       0.68      0.78      0.72        54

  

/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Best parameters for aspect 'ovr_sent': {'clf__C': 0.1, 'clf__max_iter': 100, 'clf__penalty': 'l2', 'clf__solver': 'saga'}
0.5873679416334981
0.7653724958122736
Average 10-fold CV test Accuracy for aspect 'ovr_sent': 0.5874
Average 10-fold CV train Accuracy for aspect 'ovr_sent': 0.7654
Final Training Accuracy: 0.7279
Final Training Classification Report:
               precision    recall  f1-score   support

           0       0.79      0.76      0.77       865
           1       0.37      0.79      0.50       217
           2       0.89      0.69      0.78       939

    accuracy                           0.73      2021
   macro avg       0.68      0.74      0.68      2021
weighted avg       0.79      0.73      0.74      2021

Test Accuracy: 0.6323
Test Classification Report:
               precision    recall  f1-score   support

           0       0.69      0.69      0.69       205
           1       0.21      0.41      0.28        58
           2       0.83      0.64      0.72    

In [ ]:
train_df = pd.read_csv("Romanised_train_data.csv")
test_df = pd.read_csv("Romanised_test_data.csv")
def train_final_models_grid_search(train_df, test_df, aspects):
    """
    For each aspect:
    1. Perform grid search on the training data to find the best parameters.
    2. Using the best parameters, perform 10-fold cross-validation,
       returning both the average accuracy and the list of fold estimators.
    3. Select the best estimator from CV (highest test score) and evaluate it on test data.
    Returns the best models along with their label encoders and classification reports.
    """
    best_models = {}
    train_reports = {}
    test_reports = {}

    # Parameter grid for the classifier
    param_grid = {
    'clf__C': [0.01, 0.1, 1, 10,100],
    'clf__penalty': ['l1', 'l2'],
    'clf__solver': ['liblinear', 'saga'],
    'clf__max_iter': [100, 200, 500]
}

    for aspect in aspects:
        print(f"\n=== Processing aspect: {aspect} ===")
        # Prepare data for current aspect
        X_train, y_train = prepare_data(train_df, aspect)
        X_test, y_test = prepare_data(test_df, aspect)

        # Encode target labels to numerical values
        le = LabelEncoder()
        y_train_encoded = le.fit_transform(y_train)
        y_test_encoded = le.fit_transform(y_test)

        # Build initial pipeline with a placeholder classifier
        pipeline = Pipeline([
            ('tfidf', TfidfVectorizer(max_features=5000,
                                      ngram_range=(1, 2),
                                      min_df=5,
                                      max_df=0.90,
                                      tokenizer=custom_tokenizer)),
            ('clf', LogisticRegression(random_state=42, class_weight='balanced'))
        ])

        # 10-fold CV strategy for grid search
        cv_strategy = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

        # Ensure there are no missing or empty texts in training set
        X_train = X_train.dropna(subset=['R_clean_text'])
        X_train = X_train[X_train['R_clean_text'].str.strip() != '']

        # --- Step 1: Grid Search ---
        grid = GridSearchCV(pipeline, param_grid, cv=cv_strategy,
                            scoring='balanced_accuracy', n_jobs=-1, return_train_score=True)
        grid.fit(X_train['R_clean_text'], y_train_encoded)
        best_params = grid.best_params_
        print(f"Best parameters for aspect '{aspect}': {best_params}")

        # --- Step 2: Build a new pipeline using the best parameters ---
        best_pipeline = Pipeline([
            ('tfidf', TfidfVectorizer(max_features=5000,
                                      ngram_range=(1, 2),
                                      min_df=5,
                                      max_df=0.90,
                                      tokenizer=custom_tokenizer)),
            ('clf', LogisticRegression(random_state=42,
                                             class_weight='balanced',
                                             C=best_params['clf__C'],
                                             penalty=best_params['clf__penalty'],
                                             solver=best_params['clf__solver'],
                                             max_iter=best_params['clf__max_iter']
                                             ))
        ])

        # --- Step 3: Perform 10-Fold Cross Validation with estimator return ---
        cv_results = cross_validate(best_pipeline, X_train['R_clean_text'], y_train_encoded,
                                    cv=StratifiedKFold(n_splits=10, shuffle=True, random_state=42),
                                    scoring='balanced_accuracy', return_train_score=True,return_estimator=True, n_jobs=-1)
        avg_cv_score_test = cv_results['test_score'].mean()
        avg_cv_score_train = cv_results['train_score'].mean()
        print(avg_cv_score_test)
        print(avg_cv_score_train)


        print(f"Average 10-fold CV test Accuracy for aspect '{aspect}': {avg_cv_score_test:.4f}")
        print(f"Average 10-fold CV train Accuracy for aspect '{aspect}': {avg_cv_score_train:.4f}")

        # --- Step 4: Select the best estimator from the CV folds ---
        best_index = np.argmax(cv_results['test_score'])
        best_cv_estimator = cv_results['estimator'][best_index]

        # Evaluate the selected estimator on the training data (optional)
        y_train_pred = best_cv_estimator.predict(X_train['R_clean_text'])
        train_acc = accuracy_score(y_train_encoded, y_train_pred)
        print("Final Training Accuracy: {:.4f}".format(train_acc))
        train_report = classification_report(y_train_encoded, y_train_pred)
        print("Final Training Classification Report:\n", train_report)

        # --- Step 5: Evaluate on Test Data ---
        y_test_pred = best_cv_estimator.predict(X_test['R_clean_text'])
        test_acc = accuracy_score(y_test_encoded, y_test_pred)
        print("Test Accuracy: {:.4f}".format(test_acc))
        test_report = classification_report(y_test_encoded, y_test_pred)
        print("Test Classification Report:\n", test_report)

        # Save results for the current aspect
        best_models[aspect.lower()] = {
            "model": best_cv_estimator,
            "label_encoder": le
        }
        train_reports[aspect.lower()] = train_report
        test_reports[aspect.lower()] = test_report

    return best_models, train_reports, test_reports
def prepare_data(df, aspect_column):
    """Prepare data for a specific aspect"""
    # Encode text to numerical values if needed
    # le = LabelEncoder()
    # Changed to return a DataFrame instead of a Series
    X = df[['R_clean_text']]
    y = df[aspect_column]
    return X, y

In [ ]:
best_models, train_reports, test_reports = train_final_models_grid_search(train_df,test_df, aspects)


=== Processing aspect: Acting ===


/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Best parameters for aspect 'Acting': {'clf__C': 10, 'clf__max_iter': 500, 'clf__penalty': 'l2', 'clf__solver': 'saga'}
0.6130620798758054
0.8437246864894339
Average 10-fold CV test Accuracy for aspect 'Acting': 0.6131
Average 10-fold CV train Accuracy for aspect 'Acting': 0.8437
Final Training Accuracy: 0.7813
Final Training Classification Report:
               precision    recall  f1-score   support

           0       0.18      0.90      0.30       131
           1       0.95      0.79      0.86      2962
           2       0.77      0.74      0.76       848

    accuracy                           0.78      3941
   macro avg       0.63      0.81      0.64      3941
weighted avg       0.88      0.78      0.82      3941

Test Accuracy: 0.7049
Test Classification Report:
               precision    recall  f1-score   support

           0       0.08      0.41      0.13        34
           1       0.90      0.75      0.82       746
           2       0.68      0.57      0.62       206


/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Best parameters for aspect 'direction': {'clf__C': 0.1, 'clf__max_iter': 200, 'clf__penalty': 'l2', 'clf__solver': 'saga'}
0.6430779466689169
0.7471698613858931
Average 10-fold CV test Accuracy for aspect 'direction': 0.6431
Average 10-fold CV train Accuracy for aspect 'direction': 0.7472
Final Training Accuracy: 0.7539
Final Training Classification Report:
               precision    recall  f1-score   support

           0       0.43      0.75      0.55       568
           1       0.95      0.76      0.84      3128
           2       0.38      0.74      0.50       245

    accuracy                           0.75      3941
   macro avg       0.59      0.75      0.63      3941
weighted avg       0.84      0.75      0.78      3941

Test Accuracy: 0.6765
Test Classification Report:
               precision    recall  f1-score   support

           0       0.30      0.59      0.40       124
           1       0.92      0.70      0.79       804
           2       0.27      0.60      0.37 

/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Best parameters for aspect 'Music': {'clf__C': 10, 'clf__max_iter': 200, 'clf__penalty': 'l2', 'clf__solver': 'saga'}
0.6960752674176557
0.8949856041024047
Average 10-fold CV test Accuracy for aspect 'Music': 0.6961
Average 10-fold CV train Accuracy for aspect 'Music': 0.8950
Final Training Accuracy: 0.9018
Final Training Classification Report:
               precision    recall  f1-score   support

           0       0.24      0.96      0.38        89
           1       0.99      0.91      0.95      3446
           2       0.81      0.83      0.82       406

    accuracy                           0.90      3941
   macro avg       0.68      0.90      0.72      3941
weighted avg       0.95      0.90      0.92      3941

Test Accuracy: 0.8600
Test Classification Report:
               precision    recall  f1-score   support

           0       0.10      0.56      0.18        18
           1       0.98      0.88      0.93       867
           2       0.68      0.78      0.73       101

  

/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Best parameters for aspect 'ovr_sent': {'clf__C': 1, 'clf__max_iter': 100, 'clf__penalty': 'l2', 'clf__solver': 'saga'}
0.642161398217507
0.796532962688978
Average 10-fold CV test Accuracy for aspect 'ovr_sent': 0.6422
Average 10-fold CV train Accuracy for aspect 'ovr_sent': 0.7965
Final Training Accuracy: 0.7742
Final Training Classification Report:
               precision    recall  f1-score   support

           0       0.79      0.79      0.79      1346
           1       0.48      0.79      0.60       567
           2       0.93      0.76      0.83      2028

    accuracy                           0.77      3941
   macro avg       0.73      0.78      0.74      3941
weighted avg       0.82      0.77      0.79      3941

Test Accuracy: 0.6826
Test Classification Report:
               precision    recall  f1-score   support

           0       0.72      0.67      0.69       348
           1       0.34      0.57      0.42       139
           2       0.84      0.73      0.78       4